In [56]:
import gensim, logging
from gensim.models.doc2vec import TaggedDocument 
from gensim.models import Doc2Vec
import random
import re 
import os 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [22]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [23]:
gmodel = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True) 

2022-01-03 08:49:51,535 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin
2022-01-03 08:50:36,699 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from GoogleNews-vectors-negative300.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-01-03T08:50:36.695297', 'gensim': '4.1.2', 'python': '3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'load_word2vec_format'}


In [24]:
gmodel['cat']

array([ 0.0123291 ,  0.20410156, -0.28515625,  0.21679688,  0.11816406,
        0.08300781,  0.04980469, -0.00952148,  0.22070312, -0.12597656,
        0.08056641, -0.5859375 , -0.00445557, -0.296875  , -0.01312256,
       -0.08349609,  0.05053711,  0.15136719, -0.44921875, -0.0135498 ,
        0.21484375, -0.14746094,  0.22460938, -0.125     , -0.09716797,
        0.24902344, -0.2890625 ,  0.36523438,  0.41210938, -0.0859375 ,
       -0.07861328, -0.19726562, -0.09082031, -0.14160156, -0.10253906,
        0.13085938, -0.00346375,  0.07226562,  0.04418945,  0.34570312,
        0.07470703, -0.11230469,  0.06738281,  0.11230469,  0.01977539,
       -0.12353516,  0.20996094, -0.07226562, -0.02783203,  0.05541992,
       -0.33398438,  0.08544922,  0.34375   ,  0.13964844,  0.04931641,
       -0.13476562,  0.16308594, -0.37304688,  0.39648438,  0.10693359,
        0.22167969,  0.21289062, -0.08984375,  0.20703125,  0.08935547,
       -0.08251953,  0.05957031,  0.10205078, -0.19238281, -0.09

In [25]:
gmodel['dog']

array([ 5.12695312e-02, -2.23388672e-02, -1.72851562e-01,  1.61132812e-01,
       -8.44726562e-02,  5.73730469e-02,  5.85937500e-02, -8.25195312e-02,
       -1.53808594e-02, -6.34765625e-02,  1.79687500e-01, -4.23828125e-01,
       -2.25830078e-02, -1.66015625e-01, -2.51464844e-02,  1.07421875e-01,
       -1.99218750e-01,  1.59179688e-01, -1.87500000e-01, -1.20117188e-01,
        1.55273438e-01, -9.91210938e-02,  1.42578125e-01, -1.64062500e-01,
       -8.93554688e-02,  2.00195312e-01, -1.49414062e-01,  3.20312500e-01,
        3.28125000e-01,  2.44140625e-02, -9.71679688e-02, -8.20312500e-02,
       -3.63769531e-02, -8.59375000e-02, -9.86328125e-02,  7.78198242e-03,
       -1.34277344e-02,  5.27343750e-02,  1.48437500e-01,  3.33984375e-01,
        1.66015625e-02, -2.12890625e-01, -1.50756836e-02,  5.24902344e-02,
       -1.07421875e-01, -8.88671875e-02,  2.49023438e-01, -7.03125000e-02,
       -1.59912109e-02,  7.56835938e-02, -7.03125000e-02,  1.19140625e-01,
        2.29492188e-01,  

In [26]:
gmodel['spatula']

array([-0.19140625, -0.04296875,  0.27539062,  0.00488281, -0.3203125 ,
        0.08203125,  0.05566406, -0.03613281, -0.31445312,  0.10693359,
       -0.359375  ,  0.29882812,  0.02331543,  0.05517578, -0.140625  ,
        0.1953125 , -0.23632812, -0.22167969, -0.06542969, -0.3359375 ,
        0.25195312, -0.09326172,  0.54296875,  0.11328125, -0.28710938,
       -0.12011719, -0.11181641,  0.20996094, -0.33203125,  0.30273438,
       -0.3359375 , -0.12255859,  0.12890625, -0.28515625, -0.04223633,
        0.25585938,  0.3203125 ,  0.07177734,  0.19042969, -0.01379395,
        0.16992188, -0.22460938,  0.5078125 ,  0.08398438, -0.07519531,
       -0.06396484,  0.05371094,  0.34570312,  0.46289062, -0.16699219,
       -0.30664062,  0.15234375, -0.09765625, -0.26171875, -0.14160156,
        0.2265625 ,  0.49609375, -0.10791016, -0.08447266,  0.234375  ,
        0.04931641, -0.07128906,  0.05273438, -0.11914062,  0.09814453,
        0.11181641, -0.13574219, -0.46875   ,  0.26171875,  0.12

In [27]:
gmodel.similarity('cat', 'dog')

0.76094574

In [28]:
gmodel.similarity('cat','spatula')

0.12412613

In [30]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>',' ', sent) # strip html tags 
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remove apostrophes
    sent = re.sub(r'\W',' ', sent) # remove punctuation
    sent = re.sub(r'\s+',' ', sent)# remove repeated spaces 
    sent = sent.strip()
    return sent.split()

In [31]:
# unsupervised training data 
unsup_sentences = []
# link: http://ai.stanford.edu/~amaas/data/sentiment/, data mentah dari folder IMDB
for dirname in ["train/pos", "train/neg", "train/unsup", "test/pos", "test/neg"] :
    for fname in sorted(os.listdir("aclImdb/" + dirname)):
        if fname[-4:] == '.txt':
            with open("aclImdb/" + dirname + "/"+ fname, encoding = 'UTF-8') as f: 
                sent = f.read()
                words = extract_words(sent)
                unsup_sentences.append(TaggedDocument(words, [dirname + "/" + fname])) 
# link: http://w.cs.cormeld.edu/people/pabo/movie-review-data 
for dirname in ["txt_sentoken/pos", "txt_sentoken/neg"]:
    for fname in sorted(os.listdir(dirname)) :
        if fname[-4:] == '.txt':
            with open(dirname + '/' +  fname, encoding = 'UTF-8') as f:
                for i, sent in enumerate(f):
                    words = extract_words(sent)
                    unsup_sentences.append(TaggedDocument(words, ["%s/%s-%d" % (dirname, fname, i)])) 
#link: https://nip.Stanford.edu/sentiment/, data mentah dari folder Rotten Tomatoes 
with open("stanfordSentimentTreebank/original_rt_snippets.txt", encoding = 'UTF-8') as f:
    for i, line in enumerate(f): 
        words =  extract_words(sent) 
        unsup_sentences.append(TaggedDocument (words, ["rt-%d" % i]))

In [32]:
len(unsup_sentences) 

175325

In [33]:
unsup_sentences[0:1]

[TaggedDocument(words=['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', 'hig', 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', 'teachers', 'the', 'scramble', 'to', 'survive', 'financially', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', 'pomp', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', 'i', 'immediately', 'recalled', 'at', 'high', 'a', 'classic', 'line', 'inspector', 'here', 'to', 'sack', 'one', 'of', 'your', 'teachers', 'student', 'welcome', 

In [34]:
class PermuteSentences(object):
    def __init__(self, sents):
        self.sents = sents
        
    def __iter__(self):
        shuffled = list(self.sents)
        random.shuffle(shuffled)
        for sent in shuffled:
            yield sent

In [35]:
permuter = PermuteSentences(unsup_sentences) 
model = Doc2Vec(permuter, dm=0, hs=1, vector_size=50)

2022-01-03 08:54:56,925 : INFO : collecting all words and their counts
2022-01-03 08:54:57,002 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-01-03 08:54:57,214 : INFO : PROGRESS: at example #10000, processed 1389443 words (6614899/s), 45270 word types, 10000 tags
2022-01-03 08:54:57,410 : INFO : PROGRESS: at example #20000, processed 2777491 words (7109018/s), 61293 word types, 20000 tags
2022-01-03 08:54:57,608 : INFO : PROGRESS: at example #30000, processed 4179844 words (7122306/s), 73050 word types, 30000 tags
2022-01-03 08:54:57,807 : INFO : PROGRESS: at example #40000, processed 5564595 words (6945163/s), 82275 word types, 40000 tags
2022-01-03 08:54:58,015 : INFO : PROGRESS: at example #50000, processed 6977765 words (6816706/s), 90711 word types, 50000 tags
2022-01-03 08:54:58,215 : INFO : PROGRESS: at example #60000, processed 8366321 words (6927349/s), 97948 word types, 60000 tags
2022-01-03 08:54:58,419 : INFO : PROGRESS: at example #70

In [38]:
model.save('reviews.d2v')
# menyimpan model dalam bentuk *.d2v, berfungsi untuk digunakan kembali : model = Doc2Vec.load('reviews.d2v') 

2022-01-03 08:58:50,499 : INFO : Doc2Vec lifecycle event {'fname_or_handle': 'reviews.d2v', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-01-03T08:58:50.499962', 'gensim': '4.1.2', 'python': '3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'saving'}
2022-01-03 08:58:50,520 : INFO : not storing attribute cum_table
2022-01-03 08:58:51,220 : INFO : saved reviews.d2v


In [39]:
model.infer_vector(extract_words("This place is not worth your time, let alone Vegas.")) 

array([-0.10659454,  0.17071705,  0.4162439 ,  0.17032814,  0.41947207,
       -0.29911625, -0.33283713,  0.16198134,  0.47977668,  0.1125931 ,
       -0.15485187, -0.21577936, -0.34448135,  0.03173614,  0.52906454,
        0.3081789 , -0.28324458, -0.29875776,  0.2531803 , -0.36389932,
       -0.19687769, -0.08376718, -0.2797256 ,  0.14046156,  0.1609032 ,
       -0.344272  , -0.04298849, -0.20050141, -0.14698803,  0.1811585 ,
        0.02728516,  0.22156352, -0.24114986, -0.09584662, -0.20895603,
        0.6046853 , -0.11288768,  0.30327573, -0.01637424,  0.17216153,
        0.09758569, -0.28964937,  0.02188035, -0.37003163,  0.11342382,
        0.42888105,  0.30559513, -0.27117598, -0.0165049 , -0.28815496],
      dtype=float32)

In [40]:
cosine_similarity(
    [model.infer_vector(extract_words("This place is not worth your time, let alone Vegas."))],
    [model.infer_vector(extract_words("Service sucks."))])

array([[0.3249038]], dtype=float32)

In [41]:
cosine_similarity(
    [model.infer_vector(extract_words("Highly recommended."))],
    [model.infer_vector(extract_words("Service sucks."))])

array([[0.2377208]], dtype=float32)

In [48]:
sentences = []
sentvecs = []
sentiments = []
for fname in ["yelp", "amazon_cells", "imdb"]: 
    with open("sentiment labelled sentences/%s_labelled.txt" % fname, encoding='UTF-8') as f:
        for i, line in enumerate(f):
            line_split = line.strip().split('\t')
            sentences.append(line_split[0])
            words = extract_words(line_split[0])
            sentvecs.append(model.infer_vector(words, epochs=10)) # create a vector for this document
            sentiments.append(int(line_split[1]))
            
# membuat fungsi acak dengan kombinasi setences, setvecs, dam stiments
combined = list(zip(sentences, sentvecs, sentiments))
random.shuffle(combined)
sentences, sentvecs, sentiments = zip(*combined)

In [49]:
clf = KNeighborsClassifier(n_neighbors=9)
clfrf = RandomForestClassifier()

In [50]:
scores = cross_val_score(clf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores) 

(0.788, 0.016546231527987794)

In [51]:
scores = cross_val_score(clfrf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.8063333333333332, 0.021380156947766092)

In [52]:
# kompile pembandingan kata dengan RadomForestClassifier
pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), RandomForestClassifier())

In [53]:
scores = cross_val_score(clfrf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.7953333333333334, 0.02003885115358551)